In [0]:
from pyspark.sql.functions import split, regexp_extract, udf, lit, regexp_replace
from pyspark.sql.types import IntegerType, ArrayType
from pyspark.ml.feature import OneHotEncoder, VectorAssembler

In [0]:
#directors = spark.read.csv('/FileStore/tables/rita/900.csv', header=True, inferSchema=True)
#display(directors)
#directors.printSchema()
#directors0 = spark.sql('select name as director_name,`Total awards` as total_awards from directors')
#display(directors0)
# 893 directors
# directors0.count()

In [0]:
directors = spark.sql('select director_name,year,category,outcome,original_language from directors_k')
awards_num = directors.groupby('director_name').count() \
                      .withColumnRenamed('count', 'award')
display(awards_num)
# 29503 directors
# directors.select('director_name').distinct().count()

In [0]:
# 329044 movies
details = spark.sql('select * from details')
casting = spark.sql('select * from casting')

In [0]:
# data2.write.saveAsTable('movie_genres')

In [0]:
display(details.select('title','release_date','overview','genres'))

In [0]:
movie_final = spark.sql('select * from movie_final')
display(movie_final.select('overview_vector'))

In [0]:
# 261404 movies
movies = details.filter("status = 'Released'") \
                .join(casting, 'id') \
                .select('title', 'director_name', 'director_number', 'actor_number', 'budget', 'popularity', 'release_date', 'revenue', 'runtime', 'production_companies_number', 'production_countries_number', 'spoken_languages_number', 'genres', 'vote_average', 'actor1_name', 'actor2_name', 'actor3_name', 'actor4_name', 'actor5_name', 'overview', 'tagline') \
                .filter('runtime is not null') \
                .filter('release_date is not null') \
                .filter('popularity is not null') \
                .filter('budget is not null') \
                .filter('director_number is not null') \
                .withColumnRenamed('production_companies_number', 'company') \
                .withColumnRenamed('production_countries_number', 'country') \
                .withColumnRenamed('spoken_languages_number', 'language')

In [0]:
# data2 = data2.drop('genres')

In [0]:
data = movies.join(awards_num,'director_name') \
             .select('title', 'director_number', 'actor_number', 'budget', 'popularity', 'release_date', 'revenue', 'runtime', 'company', 'country', 'language', 'genres', 'award', 'vote_average', 'actor1_name','actor2_name', 'actor3_name', 'actor4_name', 'actor5_name', 'overview', 'tagline')

In [0]:
# release_date to day, month, year
date_split = split(data['release_date'], '/')
data1 = data.withColumn('day', date_split.getItem(0).cast(IntegerType()))
data1 = data1.withColumn('month', date_split.getItem(1).cast(IntegerType()))
data1 = data1.withColumn('year', date_split.getItem(2).cast(IntegerType()))

In [0]:
data2 = data1.withColumn('genres1', split(data.genres, '\|', -1))

genre_num = {'Action':0, 'Crime':1, 'Romance':2, 'Thriller':3, 'Science':4, 'TV':5, 'Adventure':6, 'Foreign':7, 'Drama':8, 'War':9, 'Documentary':10, 'Family':11, 'Fantasy':12, 'History':13, 'Mystery':14, 'Animation':15, 'Music':16, 'Horror':17, 'Western':18, 'Comedy':19, 'Science Fiction':20, 'TV Movie':21}

def toNum(x):
  a = [0] * 22
  try:
    len(x)
  except:
    return a
  else:
    for i in range(len(x)):
      a[genre_num[x[i]]] = 1
    return a

genre_udf = udf(toNum, ArrayType(elementType=IntegerType()))
data2 = data2.withColumn("genres", genre_udf(data2["genres1"]))

display(data2)

In [0]:
# data2 = data1.withColumn('genre2', regexp_replace("genres", "|", " "))
# one = data2.withColumn('g', split(data1['genres'], "|").getItem(0))
# data2 = data1.withColumn('genre1', regexp_extract("genres", "[A-Za-z]*(?=|)", 0))
# display(data2)
# from pyspark.ml.feature import Tokenizer
# tokenizer = Tokenizer(outputCol="words")
# tokenizer.setInputCol("genres")
# tokenizer.transform(data1).head()

In [0]:
# data2.write.saveAsTable('movie_genres')

In [0]:
# one-hot company, country, language
# data4 = OneHotEncoder(inputCol="company", outputCol="company1").fit(data2).transform(data3)
# data5 = OneHotEncoder(inputCol="country", outputCol="country1").fit(data2).transform(data4)
# data6 = OneHotEncoder(inputCol="language", outputCol="language1").fit(data2).transform(data5)

# fill budget and revenue
data7 = data2.na.fill(0)

In [0]:
# delete budget, revenue == 0
# 4935 movies
# data7 = data7.filter('budget != 0').filter('revenue != 0')

In [0]:
data8 = data7.select('title', 'day', 'month', 'year', data7['genres'], 'director_number', 'actor_number', 'budget', 'popularity', 'revenue', 'runtime', data7['company'], data7['country'], data7['language'], 'award', 'vote_average', 'overview', 'tagline')
display(data8.describe())

In [0]:
data8 = data8.filter('vote_average != 0.0')

In [0]:
# data8.write.saveAsTable('movie2')

In [0]:
actorScore = spark.sql('select actor,score from actorscore')

# score = actorScore.select('score').where(actorScore['actor'] == 'none')
# display(score)

In [0]:
data9 = data7.select('title', 'day', 'month', 'year', 'genres', 'director_number', 'actor_number', 'budget', 'popularity', 'revenue', 'runtime', data7['company'], data7['country'], data7['language'], 'award', 'vote_average', 'actor1_name', 'actor2_name', 'actor3_name', 'actor4_name', 'actor5_name', 'overview', 'tagline')
data9 = data9.filter('vote_average != 0.0')
data_join = data9.join(actorScore, data9['actor1_name'] == actorScore['actor'], 'left')\
                 .withColumnRenamed('score', 'score1')\
                 .drop('actor')
# display(data_join)
                 
data_join = data_join.join(actorScore, data_join['actor2_name'] == actorScore['actor'], 'left')\
                     .withColumnRenamed('score', 'score2')\
                     .drop('actor')
data_join = data_join.join(actorScore, data_join['actor3_name'] == actorScore['actor'], 'left')\
                     .withColumnRenamed('score', 'score3')\
                     .drop('actor')
data_join = data_join.join(actorScore, data_join['actor4_name'] == actorScore['actor'], 'left')\
                     .withColumnRenamed('score', 'score4')\
                     .drop('actor')
data_join = data_join.join(actorScore, data_join['actor5_name'] == actorScore['actor'], 'left')\
                     .withColumnRenamed('score', 'score5')\
                     .drop('actor')
# data9.count()

# data_join.count()
# 69489
data_join = data_join.na.fill(0)

count_udf = udf(lambda x:(x!=0) * 1)

#62936
data10 = data_join.select('title', 'day', 'month', 'year', 'genres', 'director_number', 'actor_number', 'budget', 'popularity', 'revenue', 'runtime', 'company', 'country', 'language', 'award', 'vote_average', (data_join['score1']+ data_join['score2']+ data_join['score3']+data_join['score4']+data_join['score5']).alias('actorScore'), (count_udf(data_join['score1']) + count_udf(data_join['score2']) + count_udf(data_join['score3']) + count_udf(data_join['score4']) + count_udf(data_join['score5'])).alias('actorCount'), 'overview', 'tagline')

data10 = data10.filter('actorScore != 0')

data10 = data10.select('title', 'day', 'month', 'year', 'genres', 'director_number', 'actor_number', 'budget', 'popularity', 'revenue', 'runtime', 'company', 'country', 'language', 'award', (data10['actorScore']/data10['actorCount']).alias('actorAvgScore').astype('float'), 'vote_average', 'overview', 'tagline')
data10.count()
#69489
# display(data10)


In [0]:
display(data10)

In [0]:
# genres: array --> dense vector
from pyspark.ml.linalg import VectorUDT, Vectors, Vector

genre_udf = udf(lambda x: Vectors.dense(x), VectorUDT())

data11 = data10.withColumn('genres1', genre_udf(data10['genres']))
data11 = data11.drop('genres')
display(data11)

In [0]:
# overview and tagline fill na

data12 = data11.na.fill({'overview': '', 'tagline': ''})
display(data12)

In [0]:
print(data12.filter('overview is null').count())
print(data12.filter('tagline is null').count())

In [0]:
# Word2Vec on overview
from pyspark.ml.feature import Word2Vec, Tokenizer

tokenizer_overview = Tokenizer(inputCol='overview', outputCol="overview_array")
data13 = tokenizer_overview.transform(data12).drop('overview')

word2vec_overview = Word2Vec(vectorSize=8, minCount=0, inputCol="overview_array", outputCol="overview_vector")
model = word2vec_overview.fit(data13)

data14 = model.transform(data13).drop('overview_array')

display(data14)

In [0]:
# Word2Vec on tagline

tokenizer_tagline = Tokenizer(inputCol='tagline', outputCol="tagline_array")
data15 = tokenizer_tagline.transform(data14).drop('tagline')

word2vec_tagline = Word2Vec(vectorSize=8, minCount=0, inputCol="tagline_array", outputCol="tagline_vector")
model = word2vec_tagline.fit(data15)

data16 = model.transform(data15).drop('tagline_array')

display(data16)

In [0]:
# data17.write.saveAsTable('movie_final')

In [0]:
display(data17.filter(data17['title'] == 'Sunday in August'))

In [0]:
details = spark.sql('select original_language from details')
country = spark.sql('select * from country')
country = country.withColumnRenamed('Language', 'original_language')

In [0]:
country_language = details.join(country,'original_language').groupby('Country').count()

In [0]:
details = spark.sql('select original_language from details')
country = spark.sql('select * from country')
country = country.withColumnRenamed('Language', 'original_language')
country_language = details.join(country,'original_language').groupby('Country').count()

display(country_language)

Country,count
POL,1362
LVA,195
BRA,6847
FRA,14736
BRN,118
ETH,5
ITA,7749
UKR,207
MLI,12
AFG,3
